In [1]:
import emoji

In [2]:
emoji.EMOJI_UNICODE[":grinning_face_with_big_eyes:"]

'😃'

In [3]:
emoji_dictionary = {
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":disappointed_face:",
    "4":":fork_and_knife:"
}

In [4]:
import pandas as pd
import numpy as np

In [5]:
dfT = pd.read_csv('./train_emoji.csv',header=None)

In [6]:
dfT

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN
...,...,...,...,...
127,he had to make a home run,1,NaN,NaN
128,I am ordering food,4,NaN,NaN
129,What is wrong with you,3,NaN,NaN
130,I love you,0,NaN,NaN


In [7]:
X = dfT.values[:,0]
X.shape

(132,)

In [8]:
Y = dfT.values[:,1]
Y.shape

(132,)

In [9]:
for i in range(10):
    print(X[i],emoji.emojize(emoji_dictionary[str(Y[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😃
The assignment is too long  😞
I want to go play ⚾


In [10]:
embedding_idx = {}

In [11]:
with open('./glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        value = line.split()
        word= value[0]
        coef= np.asarray(value[1:],dtype='float')
        embedding_idx[word] = coef
        
        

In [12]:
emdIdx = embedding_idx[':'].shape[0]
print(emdIdx)

50


In [13]:
def embedding_output(X):
    max_len = 10
    embedding_out = np.zeros((X.shape[0],max_len,emdIdx))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            try:
                embedding_out[ix][jx] = embedding_idx[X[ix][jx].lower()]
            except:
                embedding_out[ix][jx] = np.zeros((50,))
    return embedding_out

In [14]:
embedding_mat_train = embedding_output(X)

In [15]:
X[10]

['she', 'did', 'not', 'answer', 'my', 'text']

In [16]:
X[12]

['how', 'many', 'points', 'did', 'he', 'score']

In [17]:
embedding_mat_train.shape

(132, 10, 50)

## **Simple LSTM Model**

In [18]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Activation,Dropout

Using TensorFlow backend.


In [19]:
m = Sequential()

In [20]:
m.add(LSTM(64,input_shape=(10,50),return_sequences=True))
m.add(Dropout(0.5))
m.add(LSTM(64,return_sequences=False))
m.add(Dense(5))
m.add(Activation('softmax'))
m.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [21]:
from keras.utils import to_categorical

In [22]:
Y_train = to_categorical(Y)
Y_train.shape

(132, 5)

In [23]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [24]:
checkpt = ModelCheckpoint(save_best_only=True,monitor='val_loss',mode='auto',period=1,filepath="best_model.h5")
early = EarlyStopping(monitor='val_loss',patience=100)

In [25]:
hist = m.fit(embedding_mat_train,Y_train,epochs=500,batch_size=64,shuffle=False,validation_split=0.2,callbacks=[checkpt,early])


Train on 105 samples, validate on 27 samples
Epoch 1/500
105/105 [==============================] - 0s 5ms/step - loss: 1.6002 - acc: 0.2667 - val_loss: 1.6154 - val_acc: 0.2963
Epoch 2/500
105/105 [==============================] - 0s 237us/step - loss: 1.5473 - acc: 0.3333 - val_loss: 1.6261 - val_acc: 0.2963
Epoch 3/500
105/105 [==============================] - 0s 247us/step - loss: 1.5019 - acc: 0.4000 - val_loss: 1.6537 - val_acc: 0.2593
Epoch 4/500
105/105 [==============================] - 0s 266us/step - loss: 1.4755 - acc: 0.4190 - val_loss: 1.6920 - val_acc: 0.2593
Epoch 5/500
105/105 [==============================] - 0s 247us/step - loss: 1.4485 - acc: 0.4667 - val_loss: 1.7219 - val_acc: 0.2222
Epoch 6/500
105/105 [==============================] - 0s 247us/step - loss: 1.4259 - acc: 0.4381 - val_loss: 1.7161 - val_acc: 0.2593
Epoch 7/500
105/105 [==============================] - 0s 266us/step - loss: 1.3863 - acc: 0.4762 - val_loss: 1.6722 - val_acc: 0.2593
Epoch 8/500

105/105 [==============================] - 0s 218us/step - loss: 0.0181 - acc: 1.0000 - val_loss: 1.5120 - val_acc: 0.5926
Epoch 60/500
105/105 [==============================] - 0s 218us/step - loss: 0.0171 - acc: 1.0000 - val_loss: 1.5184 - val_acc: 0.5926
Epoch 61/500
105/105 [==============================] - 0s 199us/step - loss: 0.0148 - acc: 1.0000 - val_loss: 1.5337 - val_acc: 0.5926
Epoch 62/500
105/105 [==============================] - 0s 209us/step - loss: 0.0188 - acc: 1.0000 - val_loss: 1.5373 - val_acc: 0.5926
Epoch 63/500
105/105 [==============================] - 0s 209us/step - loss: 0.0135 - acc: 1.0000 - val_loss: 1.5500 - val_acc: 0.6296
Epoch 64/500
105/105 [==============================] - 0s 209us/step - loss: 0.0127 - acc: 1.0000 - val_loss: 1.5569 - val_acc: 0.6296
Epoch 65/500
105/105 [==============================] - 0s 199us/step - loss: 0.0142 - acc: 1.0000 - val_loss: 1.5757 - val_acc: 0.6667
Epoch 66/500
105/105 [==============================] - 0s 19

105/105 [==============================] - 0s 256us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 1.8656 - val_acc: 0.6296
Epoch 120/500
105/105 [==============================] - 0s 275us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 1.8613 - val_acc: 0.5926


In [26]:
m.load_weights('best_model.h5')

In [27]:
m.evaluate(embedding_mat_train,Y_train)

132/132 [==============================] - 0s 106us/step


[0.6068259983351736, 0.7727272510528564]

In [28]:
dft = pd.read_csv('./test_emoji.csv',header=None)


In [29]:
Xt = dft.values[:,0]
Yt = dft.values[:,1]


In [30]:
X_test = embedding_output(Xt)

In [31]:
Y_test = to_categorical(Yt)

In [32]:
Y_test.shape

(56, 5)

In [33]:
m.evaluate(X_test,Y_test)

56/56 [==============================] - 0s 125us/step


[1.370129840714591, 0.4821428656578064]

In [34]:
pred = m.predict_classes(X_test)

In [35]:
for i in range(30):
    print(' '.join(Xt[i]))
    print("prediction:",emoji.emojize(emoji_dictionary[str(pred[i])]))
    print("Actuall:",emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(" ")

I want to eat
prediction: 🍴
Actuall: 🍴
 
he did not answer
prediction: 😞
Actuall: 😞
 
he got a raise
prediction: ⚾
Actuall: 😃
 
she got me a present
prediction: 😞
Actuall: ❤️
 
ha ha ha it was so funny
prediction: 😃
Actuall: 😃
 
he is a good friend
prediction: 😃
Actuall: ❤️
 
I am upset
prediction: 😞
Actuall: ❤️
 
We had such a lovely dinner tonight
prediction: 😃
Actuall: ❤️
 
where is the food
prediction: 🍴
Actuall: 🍴
 
Stop making this joke ha ha ha
prediction: 😃
Actuall: 😃
 
where is the ball
prediction: ⚾
Actuall: ⚾
 
work is hard
prediction: 😃
Actuall: 😞
 
This girl is messing with me
prediction: ❤️
Actuall: 😞
 
are you serious ha ha
prediction: 😞
Actuall: 😃
 
Let us go play baseball
prediction: ⚾
Actuall: ⚾
 
This stupid grader is not working
prediction: 😞
Actuall: 😞
 
work is horrible
prediction: 😃
Actuall: 😞
 
Congratulation for having a baby
prediction: 😃
Actuall: 😃
 
stop messing around
prediction: 🍴
Actuall: 😞
 
any suggestions for dinner
prediction: 😃
Actuall: 🍴
 
I love ta

In [36]:
with open("model.json", "w") as file:
    file.write(m.to_json())
m.save_weights("model.h5")

In [37]:
from keras.models import model_from_json

In [38]:
with open("model.json",'r') as f:
    m = model_from_json(f.read())
m.load_weights("model.h5")

In [39]:
test_str = input("Enter some text: ")
X = pd.Series([test_str])

Enter some text: happy


In [40]:
X

0    happy
dtype: object

In [41]:
emb_X = embedding_output(X)

In [42]:
X

0    [happy]
dtype: object

In [43]:
p = m.predict_classes(emb_X)

In [44]:

print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

happy
😃
